In [1]:
import os
import json
import numpy as np

from csbdeep.models import Config, CARE
from matplotlib import pyplot as plt
import tifffile as tiff

from flame.engine import CAREInferenceSession
from flame import FLAMEImage
from flame.error import FLAMEImageError
from flame.utils import min_max_norm

2025-06-17 14:45:38.168751: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-17 14:45:38.176112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750196738.184479 1618023 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750196738.186903 1618023 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750196738.193509 1618023 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# INFERENCE_DATA_DIR = "/mnt/d/data/raw/0013_250514_HS6307_CAREtest_NA"
INFERENCE_DATA_DIR = "/mnt/d/data/raw/S268_250409_CAREtraining_PL"
# INFERENCE_OUTPUT = "/mnt/d/data/output/0013_250514_HS6307_CAREtest_NA"
INFERENCE_OUTPUT = "/mnt/d/data/output/S268_250409_CAREtraining_PL"
DATASET_DIRECTORY = "/mnt/d/code/Balu_CARE/datasets"
DATASET_NAME = "20250513_40I_denoising_7to40F"
DATASET_JSON = os.path.join(DATASET_DIRECTORY, f"{DATASET_NAME}.json")
MODEL_DIRECTORY = "/mnt/d/models/CARE/test_model"
MODEL_NAME = os.path.basename(MODEL_DIRECTORY)
INFERENCE_OUTPUT_DIRECTORY = os.path.join(INFERENCE_OUTPUT, MODEL_NAME)

ONNX_PATH = os.path.join(MODEL_DIRECTORY, f"{MODEL_NAME}.onnx")
JSON_PATH = os.path.join(MODEL_DIRECTORY, f"model_config.json")

for f in [DATASET_JSON, ONNX_PATH, JSON_PATH]:
    assert os.path.isfile(f)

In [3]:
os.makedirs(INFERENCE_OUTPUT_DIRECTORY, exist_ok = True)

In [4]:
images = []
for root, dirs, files in os.walk(INFERENCE_DATA_DIR):
    for f in files:
        if "tif" in f or "tiff" in f:
            try:
                this_image = FLAMEImage(
                    impath = os.path.join(root, f),
                    jsonext = "tileData.txt",
                    # overrideNFrames = 1,
                    checkFrames = True,
                    checkZs = False,
                    requireBidirectionalCorrection=False
                )
            except FLAMEImageError as e:
                continue
            images.append(this_image)

'bidirectionalCorrection' could not be loaded from tile data JSON.
MSG: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'
<tifffile.TiffFile 'Im_00001.tif'> <asarray> failed to reshape (3, 1200, 1200) to (40, 3, 1200, 1200), raised ValueError('cannot reshape array of size 4320000 into shape (40,3,1200,1200)')
'bidirectionalCorrection' could not be loaded from tile data JSON.
MSG: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'
<tifffile.TiffFile 'Im_00002.tif'> <asarray> failed to reshape (3, 1200, 1200) to (40, 3, 1200, 1200), raised ValueError('cannot reshape array of size 4320000 into shape (40,3,1200,1200)')
'bidirectionalCorrection' could not be loaded from tile data JSON.
MSG: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'
<tifffile.TiffFile 'Im_00003.tif'> <asarray> failed to reshape (3, 1200, 1200) to (40, 3, 1200, 1200), raised ValueError('cannot reshape array of si

In [5]:
len(images)

31

In [6]:
shapes = []
axes = []
for im in images:
    if im.imShape not in shapes:
        shapes.append(im.imShape)
    if im.axes_shape not in axes:
        axes.append(im.axes_shape)

In [7]:
shapes

[(3, 1200, 1200), (3, 7700, 1100), (40, 3, 1200, 1200), (31, 1200, 1200)]

In [8]:
axes

['CYX', 'FCYX']

In [9]:
# model = CARE(config=None, name=MODEL_DIRECTORY)
engine = CAREInferenceSession(
    model_path=ONNX_PATH,
    model_config_path=JSON_PATH,
    dataset_config_path=JSON_PATH,
)

In [10]:
print(images[10].axes_shape, images[10].imShape)
print(images[20].axes_shape, images[20].imShape)
print(images[30].axes_shape, images[30].imShape)

CYX (3, 7700, 1100)
FCYX (40, 3, 1200, 1200)
CYX (31, 1200, 1200)


In [11]:
pred_CYX = engine.predict_FLAME(images[10])
pred_FCYX = engine.predict_FLAME(images[20])
pred_CYX2 = engine.predict_FLAME(images[30])

[0, 2, 3, 1]
(np.int64(1), 7700, 1100, 3)


Could not infer on array of shape (549, 128, 128, 1) and dtype float32.
ERROR: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: patch for the following indices
 index: 3 Got: 1 Expected: 3
 Please fix either the inputs/outputs or the model.


CAREInferenceError: Could not infer on array of shape (549, 128, 128, 1) and dtype float32.
ERROR: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: patch for the following indices
 index: 3 Got: 1 Expected: 3
 Please fix either the inputs/outputs or the model.

In [ ]:
images[20].get_frames((0,5)).shape

In [ ]:
test_im = images[0].raw()[0,...].transpose(1,2,0)

In [ ]:
plt.imshow(test_im)

In [ ]:
patches = engine._get_patches(test_im)

In [ ]:
patches.shape

In [ ]:
images[0].imShape

In [ ]:
# figure = plt.figure(figsize=(10, 10))
# for pdx, patch in enumerate(patches):
#     ax = figure.add_subplot(10, 10, pdx+1)
#     ax.imshow(patch)
#     ax.set_axis_off()


In [ ]:
restitched = engine._stitch_patches(patches, (1200,1200,3))

In [ ]:
np.all((test_im == restitched))

In [ ]:
from flame.utils import min_max_norm
import json

In [ ]:
config = json.load(open(JSON_PATH, 'r'))

In [ ]:
input_1pct = config["FLAME_Dataset"]["input"]["pixel_1pct"]
input_99pct = config["FLAME_Dataset"]["input"]["pixel_99pct"]
output_1pct = np.array(config["FLAME_Dataset"]["output"]["pixel_1pct"])
output_99pct = np.array(config["FLAME_Dataset"]["output"]["pixel_99pct"])

In [ ]:
patches = np.clip(patches, a_min=input_1pct, a_max=output_99pct)

In [ ]:
patches = min_max_norm(patches, mini=input_1pct, maxi=input_99pct)

In [ ]:
pred = engine.predict(patches.astype(np.float32))[0]

In [ ]:
pred.dtype

In [ ]:
denoised = engine._stitch_patches(pred, (1200,1200,3))

In [ ]:
denoised = denoised * (output_99pct - output_1pct) + output_1pct

In [ ]:
denoised = (denoised - denoised.min()) / (denoised.max() - denoised.min())

In [ ]:
denoised.max()

In [ ]:
plt.imshow(denoised)

In [ ]:
dataset_dict = json.load(open(DATASET_JSON, 'r'))

In [ ]:
input_min = np.array(dataset_dict['FLAME_Dataset']['input']['pixel_1pct'])
input_max = np.array(dataset_dict['FLAME_Dataset']['input']['pixel_99pct'])
output_min = np.array(dataset_dict['FLAME_Dataset']['output']['pixel_1pct'])
output_max = np.array(dataset_dict['FLAME_Dataset']['output']['pixel_99pct'])

In [ ]:
for image in images:
    image.openImage()
    model_input = image.imageData.copy()
    image.closeImage()
    model_input = model_input.transpose(0, 2, 3, 1)
    model_input = np.clip(model_input, input_min, input_max)
    model_input = min_max_norm(model_input, input_min, input_max, dtype=np.float32)
    outputs = []
    for zdx in range(model_input.shape[0]):
        outputs.append(model.predict(model_input[zdx,...], axes='YXC'))
    outputs = np.stack(outputs, axis = -1).transpose(3, 0, 1, 2)
    outputs = (outputs * (output_max - output_min + 1e-20)) + output_min
    outputs[...,0] *= 4/3
    outputs[...,2] *= 2/3
    imname = "_".join(image.impath.split(os.path.sep)[4:])
    os.makedirs(os.path.join(INFERENCE_OUTPUT_DIRECTORY, os.path.splitext(imname)[0]))
    tiff.imwrite(os.path.join(INFERENCE_OUTPUT_DIRECTORY, imname), outputs)

In [ ]:
DIR = "/mnt/c/Users/BaluLab/Desktop/raw_vis"
os.makedirs(DIR)

In [ ]:
for idx, image in enumerate(images):
    image.openImage()
    raw = image.imageData.copy()
    image.closeImage()
    raw = raw.transpose(0, 2, 3, 1).astype(np.float32)
    raw[...,0] *= 4/3
    raw[...,2] *= 2/3
    raw = min_max_norm(raw, np.min(raw), np.max(raw), dtype=np.float32)
    raw = (raw * 255).astype(np.uint8)
    for zdx in [0, 4, 9]:
        impath = os.path.join(DIR, f"im_i{idx}_z{zdx}.tif")
        tiff.imwrite(os.path.join(impath), raw[zdx,...])
